In [ ]:
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img #import image need this
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import layers
from keras.layers import Activation, Dropout, Flatten, Dense,Input,BatchNormalization,AveragePooling2D
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D,Conv2D
from keras.applications import VGG16
from keras.applications import VGG19
from keras.applications import ResNet50
from keras.applications import MobileNetV2
from keras.callbacks import ModelCheckpoint
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras import optimizers
import  tensorflow as  tf
import keras.backend as K
import os
import random
import scipy
from keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix,roc_curve,roc_auc_score,auc

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
def history_plot(results,img_name): 
    # list all data in history
    print(results.history.keys())
    # summarize history for accuracy
    plt.figure(1)
    plt.plot(results.history['acc'])
    plt.plot(results.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.savefig('Accuracy_'+img_name,dpi=300)
    plt.show()

    # summarize history for loss
    plt.figure(2)
    plt.plot(results.history["loss"], label="loss")
    plt.plot(results.history["val_loss"], label="val_loss")
    plt.plot(np.argmin(results.history["val_loss"]), 
             np.min(results.history["val_loss"]), 
             marker="x", color="r", label="best model")
    plt.xlabel("Epochs")
    plt.ylabel("log_loss")
    plt.legend();
    plt.savefig('Loss_'+img_name ,dpi=300)
    plt.show()

In [ ]:
#flow_from_directory目錄需要分3類
train_image = './pneu_dataset/train'
valid_image = './pneu_dataset/validation'
test_image = './pneu_dataset/test'

In [ ]:
# Setting the default value
# dimensions of our images.
img_width, img_height = 229, 229
batch_size = 32
epochs = 700
seed = 42
class_mode = 'categorical'
date = '2019-06-23_resNet'

In [ ]:
#ImageDataGenerator
# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255,
                             rotation_range = 180)  #圖片隨機轉動
#                              width_shift_range = 0.2, #圖片水平偏移
#                              height_shift_range = 0.2, #圖片垂直偏移
#                              zoom_range = 0.3)

# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
       train_image,
        target_size = (img_width, img_height),
        batch_size = batch_size,
        shuffle=True,
        class_mode = class_mode,
        seed = seed)

validation_generator = datagen.flow_from_directory(
        valid_image,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode = class_mode,
        shuffle = False ,
        seed = seed)

# test_generator = datagen.flow_from_directory(
#         test_image,
#         target_size = (img_width, img_height),
#         batch_size = 1,
#         class_mode = None,
#         shuffle = False,
#         seed = seed
#         )

In [ ]:
# #resent50架構
# def conv_block(input_tensor, kernel_size, filters, stage, block, strides):
#     """conv_block is the block that has a conv layer at shortcut
#     # Arguments
#         input_tensor: input tensor 等於x
#         kernel_size: (3x3大小) defualt 3, the kernel size of middle conv layer at main path
#         filters: (利用list傳入) list of integers, the filterss of 3 conv layer at main path
#         stage: integer, current stage label, used for generating layer names
#         block: 'a','b'..., current block label, used for generating layer names 
#     # Returns
#         Output tensor for the block.
#     Note that from stage 3, the first conv layer at main path is with strides=(2,2)
#     And the shortcut should have strides=(2,2) as well
#     """
#     #串列把數值給3個變數
#     filters1, filters2, filters3 = filters
#     if K.image_data_format() == 'channels_last':
#         bn_axis = 3
#     else:
#         bn_axis = 1
#     conv_name_base = 'res' + str(stage) + block + '_branch'
#     bn_name_base = 'bn' + str(stage) + block + '_branch'
 
#     x = Conv2D(filters1, kernel_size=(3, 3), strides=strides, padding='same',name=conv_name_base + '2a')(input_tensor)             
#     x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
#     x = Activation('relu')(x)
 
#     x = Conv2D(filters2, kernel_size=(3, 3),strides=strides, padding='same',name=conv_name_base + '2b')(x)               
#     x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
#     x = Activation('relu')(x)
 
#     x = Conv2D(filters3, kernel_size=(3, 3),strides=strides,padding='same',name=conv_name_base + '2c')(x)
#     x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)
 
#     shortcut = Conv2D(filters3, kernel_size=(3, 3), strides=strides, padding='same',name=conv_name_base + '1')(x)                     
#     shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)
 
#     #加入2個路徑 一個是正常 一個是最短路徑 電腦會自動選擇最佳路徑
#     x = layers.add([x, shortcut])
#     x = Activation('relu')(x)
#     return x

# def identity_block(input_tensor, kernel_size, filters, stage, block,strides):
#     """The identity block is the block that has no conv layer at shortcut.
#     # Arguments
#         input_tensor: input tensor  #输入变量#
#         kernel_size: defualt 3, the kernel size of middle conv layer at main path #卷积核的大小#
#         filters: list of integers, the filterss of 3 conv layer at main path  #卷积核的数目#
#         stage: integer, current stage label, used for generating layer names #当前阶段的标签#
#         block: 'a','b'..., current block label, used for generating layer names #当前块的标签#
#     # Returns
#         Output tensor for the block.  
#     """
#     filters1, filters2, filters3 = filters  #滤波器的名称#
#     if K.image_data_format() == 'channels_last':  #代表图像通道维的位置#
#         bn_axis = 3
#     else:
#         bn_axis = 1
#     conv_name_base = 'res' + str(stage) + block + '_branch'
#     bn_name_base = 'bn' + str(stage) + block + '_branch'
 
#     x = Conv2D(filters1, kernel_size=(3, 3),strides=strides,padding='same',name=conv_name_base + '2a')(input_tensor)
#     x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
#     x = Activation('relu')(x)   #卷积层，BN层，激活函数#
 
#     x = Conv2D(filters2, kernel_size=(3, 3),strides=strides,padding='same', name=conv_name_base + '2b')(x)               
#     x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
#     x = Activation('relu')(x)
 
#     x = Conv2D(filters3, kernel_size=(3, 3),strides=strides, padding='same',name=conv_name_base + '2c')(x)
#     x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)
 
#     x = layers.add([x, x])
#     x = Activation('relu')(x)
#     return x

# # model = ResNet50(include_top=False,weights='imagenet',input_shape=(img_width,img_height,3))
# input_tensor = Input(shape=(img_width, img_height, 3))
# #3x3的濾鏡大小
# x = ZeroPadding2D((3, 3))(input_tensor)
# x = Conv2D(nb_filter=64, kernel_size=(7, 7), strides=(2, 2), padding='valid', activation='relu', name='conv1')(x)
# x = BatchNormalization(axis=3, name='bn_conv1')(x)
# x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
# #stage2#
# x = conv_block(x, kernel_size=(3, 3), filters=[64, 64, 256], stage=2, block='a', strides=(1, 1))
# x = identity_block(x, kernel_size=(3, 3),filters=[64, 64, 256], stage=2, block='b',strides=(1, 1))
# x = identity_block(x, kernel_size=(3, 3),filters=[64, 64, 256], stage=2, block='c',strides=(1, 1))


# #stage3#
# x = conv_block(x, kernel_size=(3, 3), filters=[128, 128, 512], stage=3, block='a', strides=(1, 1))
# x = identity_block(x, kernel_size=(3, 3), filters=[128, 128, 512], stage=3, block='b', strides=(1, 1))
# x = identity_block(x, kernel_size=(3, 3), filters=[128, 128, 512], stage=3, block='c', strides=(1, 1))
# x = identity_block(x, kernel_size=(3, 3), filters=[128, 128, 512], stage=3, block='d', strides=(1, 1))
# #stage4#
# x = conv_block(x, kernel_size=(3, 3), filters=[256, 256, 1024], stage=4, block='a', strides=(1, 1))
# x = identity_block(x, kernel_size=(3, 3), filters=[256, 256, 1024], stage=4, block='b', strides=(1, 1))
# x = identity_block(x, kernel_size=(3, 3), filters=[256, 256, 1024], stage=4, block='c', strides=(1, 1))
# x = identity_block(x, kernel_size=(3, 3), filters=[256, 256, 1024], stage=4, block='d', strides=(1, 1))
# x = identity_block(x, kernel_size=(3, 3), filters=[256, 256, 1024], stage=4, block='e', strides=(1, 1))
# x = identity_block(x, kernel_size=(3, 3), filters=[256, 256, 1024], stage=4, block='f', strides=(1, 1))
# #stage5#
# x = conv_block(x, kernel_size=(3, 3), filters=[512, 512, 2048], stage=5, block='a', strides=(1, 1))
# x = identity_block(x, kernel_size=(3, 3), filters=[512, 512, 2048], stage=5, block='b', strides=(1, 1))
# x = identity_block(x, kernel_size=(3, 3), filters=[512, 512, 2048], stage=5, block='c', strides=(1, 1))

# x_fc = AveragePooling2D(pool_size=(7, 7))(x)
# x_fc = Flatten()(x_fc)



In [ ]:
# model_net = Model(inputs=input_tensor,outputs=x_fc)
# model_net.load_weights('./resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')
# model = Dense(3, activation='softmax')(model_net)


In [ ]:
net = ResNet50(include_top=False,weights='imagenet',input_shape=(img_width,img_height,3))
x= net.output
x = Flatten()(x)
# x = Dropout(0.5)(x)
output_layer = Dense(3, activation='softmax', name='softmax')(x)
model = Model(inputs=net.input, outputs=output_layer)

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-6),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# 設定checkpoint & callbacks
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint
def callbacks(name):
    save_dir = "checkpoint"
    #     tl.files.exists_or_mkdir(save_dir)
    path=save_dir+'/model_check_'+name+'.h5'
    callbacks = [
        EarlyStopping(patience=25, verbose=1),
        ReduceLROnPlateau(factor=0.1, patience=10, min_lr=0.000001, verbose=1),
        ModelCheckpoint(path, verbose=1, save_best_only=True, save_weights_only=False)
#         ModelCheckpoint(path,monitor='val_acc', verbose=1,
#                         save_best_only=True, save_weights_only=False)
    ]
    return callbacks

In [ ]:
num_of_train_samples  = train_generator.classes.shape[0]
num_of_test_samples = validation_generator.classes.shape[0]

In [ ]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch= num_of_train_samples//batch_size,
        validation_data=validation_generator,
        validation_steps= num_of_test_samples//batch_size,
        epochs = epochs,
        callbacks=callbacks(date+'_fit'))

In [ ]:
history_plot(history,'resNet50.png')

In [ ]:
# Load CheckPoint
from keras.models import load_model
model= load_model('./checkpoint/model_check_2019-06-23_resNet_fit.h5')

In [ ]:
# evaluate the model
model.evaluate_generator(generator=validation_generator,
#                          steps=validation_generator.classes.shape[0]//32 +1,
                         steps=validation_generator.classes.shape[0],
                         verbose=1)

In [ ]:

# predict the output
Y_pred = model.predict_generator(validation_generator,
                                 steps=num_of_test_samples//32 +1,
                                 verbose=1)

In [ ]:
# Use auc to show the performance
auc = roc_auc_score(validation_generator.classes,Y_pred)
print("AUC: {}".format(auc))
# fpr, tpr, thresholds = roc_curve(validation_generator.classes, Y_pred)

In [ ]:
# Plot ROC curve
plt.figure(3)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr,tpr,label='AUC = %0.2f' % auc)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
# plt.show()
plt.savefig("roc_curve.png",dpi=300)

In [ ]:
# Load model
# from keras.models import load_model 
# base_model= load_model('./checkpoint/model_check_2019-06-16_VGG16_fit.h5')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
plt.rcParams['figure.figsize'] = 8,8

from scipy.ndimage.interpolation import zoom
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input, decode_predictions

import os
import gradcamutils # source code

In [ ]:
def decode_predictions_cust(preds, class_custom, top=3):
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [tuple(class_custom[i][0])+(class_custom[i][1],) + (pred[i]*100,)\
                  for i in top_indices] 
        results.append(result)
    return results

In [ ]:
count = 0
class_name=[('0','Lung Opacity'),('1','Normal'),('2','Not Normal')]
label = [] # label Lung Opacity, Normal, Not Normal
# path of img 
valid_lung_op_path='./pneu_dataset/validation/Lung Opacity/'
valid_normal_path='./pneu_dataset/validation/Normal/'
valid_not_normal_path='./pneu_dataset/validation/Not Normal/'
paths = [valid_lung_op_path,valid_normal_path,valid_not_normal_path]
# list of each img
valid_path = []
# dir 
for path in paths:
    for file in  os.listdir(path):
        if path == './pneu_dataset/validation/Lung Opacity/':
            valid_path.append(path+file)
            label.append('Lung Opacity')
        elif path == './pneu_dataset/validation/Normal/':
            valid_path.append(path+file)
            label.append('Normal')
        elif path == './pneu_dataset/validation/Not Normal/':
            valid_path.append(path+file)
            label.append('Not Normal')
        else :
            label.append('None')

In [ ]:
# grad cam 
for path in valid_path:
    
    count+=1
    if count <= 0:
        pass
    else :
        orig_img = np.array(load_img(path,target_size=(229,229)),dtype=np.uint8)
        img = np.array(load_img(path,target_size=(229,229)),dtype=np.float64)
        img = np.expand_dims(img,axis=0)
        img = preprocess_input(img)
        predictions = model.predict(img)
        top_n = 3
        top = decode_predictions_cust(predictions,class_name ,top=top_n)[0]
    #     top = decode_predictions(predictions, top=top_n)[0]
        cls = np.argsort(predictions[0])[-top_n:][::-1]

        gradcam=gradcamutils.grad_cam(model,img,layer_name='res5c_branch2c')
        gradcamplus=gradcamutils.grad_cam_plus(model,img,layer_name='res5c_branch2c')
        print(path)
        print("class activation map for:",top[0])
        fig, ax = plt.subplots(nrows=1,ncols=3)
        plt.subplot(131)
        plt.imshow(orig_img)
        plt.title("input image \n"+'('+str(label[count])+')')

        plt.subplot(132)
        plt.imshow(orig_img)
        plt.imshow(gradcam,alpha=0.8,cmap="jet")
        plt.title("Grad-CAM \n"+str(top[0][1])+'\n'+str(top[0][2])[:6])

        plt.subplot(133)
        plt.imshow(orig_img)
        plt.imshow(gradcamplus,alpha=0.8,cmap="jet")
        plt.title("Grad-CAM++ \n"+str(top[0][1])+'\n'+str(top[0][2])[:6])
        plt.savefig('./grad_cam_image/resNet/resNet_'+str(count)+'.png',dpi=300)
    #     plt.show()

In [ ]:
# # grad cam  
# def grad_CAM(image_path):
#     im = load_img(image_path, target_size=(299,299))
#     x = preprocess_input(im)
#     x = base_model.predict(x)
#     pred = model.predict(x)
    
#     # Predicted class index
#     index = np.argmax(pred)
    
#     # Get the entry of the predicted class
#     class_output = model.output[:, index]
    
#     # The last convolution layer in the model
#     last_conv_layer = model.get_layer('conv2d_1')
#     # Number of channels
#     nmb_channels = last_conv_layer.output.shape[3]

#     # Gradient of the predicted class with respect to the output feature map of the 
#     # the convolution layer with nmb_channels channels
#     grads = K.gradients(class_output, last_conv_layer.output)[0] 
    
    
#     # Vector of shape (nmb_channels,), where each entry is the mean intensity of the gradient over 
#     # a specific feature-map channel”
#     pooled_grads = K.mean(grads, axis=(0, 1, 2))

#     # Setup a function to extract the desired values
#     iterate = K.function(model.inputs, [pooled_grads, last_conv_layer.output[0]])
#     # Run the function to get the desired calues
#     pooled_grads_value, conv_layer_output_value = iterate([x])
    
    
#     # Multiply each channel in the feature-map array by “how important this channel is” with regard to the 
#     # predicted class
#     for i in range(nmb_channels):
#         conv_layer_output_value[:, :, i] *= pooled_grads_value[i]
#     # The channel-wise mean of the resulting feature map is the heatmap of the class activation.
#     heatmap = np.mean(conv_layer_output_value, axis=-1)   
    
#     # Normalize the heatmap betwen 0 and 1 for visualization
#     heatmap = np.maximum(heatmap, 0)
#     heatmap /= np.max(heatmap)
    
    
#     # Read the image again, now using cv2
#     img = cv2.imread(image_path)
#     # Size the heatmap to the size of the loaded image
#     heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
#     # Convert to RGB
#     heatmap = np.uint8(255 * heatmap)
#     # Pseudocolor/false color a grayscale image using OpenCV’s predefined colormaps
#     heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    
#     # Superimpose the image with the required intensity
#     superimposed_img = heatmap * 0.5 + img   
    
#     # Write the image
#     plt.figure(figsize=(24,12))
#     cv2.imwrite('./tmp.jpg', superimposed_img)
#     plt.imshow(mpimg.imread('./tmp.jpg'))
#     plt.title(image_path)
#     plt.show()
    
    
    